In [6]:
import requests
import pandas as pd
from pandas import DataFrame 
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from numpy import nan
from pprint import pprint 
print("Import Successful")
#!wget https://www150.statcan.gc.ca/n1/tbl/csv/10100084-eng.zip
#!unzip 10100084-eng.zip

Import Successful


# Downloading the data

In [7]:



df = pd.read_csv("TorontoListings.csv")


# Data cleaning

In [8]:
df.describe()

,id,host_id,neighbourhood_group,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.874600e+04,1.874600e+04,0.0,18746.000000,18746.000000,18746.000000,18746.000000,18746.000000,14788.000000,18746.000000,18746.000000
mean,2.673976e+07,1.087340e+08,NaN,43.680717,-79.397420,135.468473,10.786568,26.635602,1.214266,4.715246,125.690761
std,1.298332e+07,1.055992e+08,NaN,0.048263,0.064453,263.761517,36.913904,52.418576,1.597121,10.354866,136.844548
min,1.419000e+03,1.565000e+03,NaN,43.586710,-79.634800,11.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,1.634553e+07,2.049040e+07,NaN,43.645170,-79.426310,61.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,2.775978e+07,6.787450e+07,NaN,43.662815,-79.397350,99.000000,2.000000,6.000000,0.600000,1.000000,83.000000
75%,3.853432e+07,1.823192e+08,NaN,43.700153,-79.376860,150.000000,7.000000,27.000000,1.580000,3.000000,252.000000
max,4.578971e+07,3.711952e+08,NaN,43.836900,-79.127810,13137.000000,1125.000000,828.000000,15.440000,88.000000,365.000000


From above, We can see there is a column which is empty, Hence, We drop empty, NAN data from the set. 

In [9]:

df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group               float64
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

Since id and host_name, both are giving the same information, We keep one of them and We will do the same thing for name and coordination(lattitude+longitude), We will concatenate them into coulumn location

In [10]:
df.drop(['neighbourhood_group', 'name', 'host_name'], 1, inplace = True)
df.head()

,id,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,1419,1565,Little Portugal,43.64617,-79.42451,Entire home/apt,469,4,7,2017-12-04,0.11,1,0
1,8077,22795,Waterfront Communities-The Island,43.64105,-79.37628,Private room,98,180,169,2013-08-27,1.24,2,365
2,12604,48239,Annex,43.66724,-79.41598,Private room,66,1,0,NaN,NaN,1,0
3,23691,93825,Briar Hill-Belgravia,43.69602,-79.45468,Private room,70,1,217,2019-12-22,1.72,2,240
4,26654,113345,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,125,21,40,2020-03-20,0.34,2,295


In [11]:
df['location'] = [', '.join(str(x) for x in y) for y in map(tuple, df[['latitude', 'longitude']].values)]
df.head(20)

,id,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,location
0,1419,1565,Little Portugal,43.64617,-79.42451,Entire home/apt,469,4,7,2017-12-04,0.11,1,0,"43.64617, -79.42451"
1,8077,22795,Waterfront Communities-The Island,43.64105,-79.37628,Private room,98,180,169,2013-08-27,1.24,2,365,"43.64105, -79.37628000000001"
2,12604,48239,Annex,43.66724,-79.41598,Private room,66,1,0,NaN,NaN,1,0,"43.66724, -79.41598"
3,23691,93825,Briar Hill-Belgravia,43.69602,-79.45468,Private room,70,1,217,2019-12-22,1.72,2,240,"43.696020000000004, -79.45468000000001"
4,26654,113345,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,125,21,40,2020-03-20,0.34,2,295,"43.6453, -79.3894"
5,27423,118124,Greenwood-Coxwell,43.66890,-79.32592,Entire home/apt,54,120,26,2011-08-30,0.21,1,0,"43.6689, -79.32592"
6,28160,86838,Mount Pleasant West,43.70376,-79.39077,Entire home/apt,50,60,7,2018-10-17,0.11,1,364,"43.703759999999996, -79.39076999999999"
7,30931,22795,Waterfront Communities-The Island,43.64151,-79.37643,Entire home/apt,131,180,1,2010-08-11,0.01,2,365,"43.64151, -79.37643"
8,40456,174063,South Parkdale,43.63532,-79.44049,Entire home/apt,100,30,110,2020-03-25,0.88,5,359,"43.63532, -79.44049"
9,41887,183071,Oakridge,43.69466,-79.28667,Entire home/apt,75,28,82,2019-09-02,1.74,2,342,"43.69466, -79.28667"


# Taking care of missing data

In [12]:
df = df[['id', 'host_id', 'neighbourhood', 'latitude', 'longitude', 'room_type','minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'location', 'price']]

In [13]:
# replace '0' values with 'nan'
df[['number_of_reviews', 'availability_365']] = df[['number_of_reviews', 'availability_365']].replace(0, nan)
# count the number of nan values in each column
print(df.isnull().sum())

id                                   0
host_id                              0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                 3958
last_review                       3958
reviews_per_month                 3958
calculated_host_listings_count       0
availability_365                  6641
location                             0
price                                0
dtype: int64


In [14]:
#conversion of the 'REF_DATE' from a string to a proper datetime object.
df['last_review'] = pd.to_datetime(df['last_review']) 
# fill missing values with mean column values
df.fillna(df.mean(), inplace=True)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  after removing the cwd from sys.path.


# Encoding categorical Data

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
X=df.iloc[: , 5:12].values
y=df.iloc[:, 13].values
one_hot = pd.get_dummies(df['room_type'])


In [16]:
one_hot_df = pd.concat([df, one_hot], axis=1)


In [17]:
one_hot_df = one_hot_df[['id', 'host_id', 'neighbourhood','minimum_nights', 'number_of_reviews',
       'calculated_host_listings_count', 'availability_365', 'reviews_per_month','latitude', 'longitude','Shared room', 'Entire home/apt',
       'Hotel room', 'Private room', 'price']]


In [18]:
#Encoding neighbourhood
one_hot_neighbourhood = pd.get_dummies(df['neighbourhood'])


In [19]:
one_hot_encoded_df = pd.concat([one_hot_df, one_hot_neighbourhood], axis=1)

one_hot_encoded_df["reviews_per_month"].mean()

filtered_df = one_hot_encoded_df[one_hot_encoded_df['reviews_per_month'] >= 1.2] 

In [20]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])
filtered_df = movecol(filtered_df, 
             cols_to_move=['Yorkdale-Glen Park','price'], 
             ref_col='York University Heights',
             place='After')


Decision Tree Reggressor
to see how the dataset works with a regression model, I applied a decision tree regressor on air bnb dataset.

In [21]:
#define dataset
X_reg=filtered_df.iloc[: , 7:-1].values
y_reg=filtered_df.iloc[:, -1].values
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size = 0.4, random_state = 0)
#define the model
Regressor = DecisionTreeRegressor()

In [24]:
Regressor.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

To check on the model accuracy, I employed cross validation score.


In [35]:
from sklearn.model_selection import cross_val_score
cross_val_score(Regressor, X_reg, y_reg, cv=10)

array([ -2.23396125,  -1.34820058,  -0.04646368,  -1.23735069,
        -0.25799366,  -1.93097007,  -0.0681571 ,  -1.68680136,
        -2.11801324, -13.62971362])

In [25]:
print(Regressor.feature_importances_)

[3.81458042e-02 7.48004380e-01 1.37814972e-01 2.25532552e-04
 1.76304452e-02 7.88857805e-05 3.19373543e-04 1.73669656e-05
 1.14062946e-04 2.17732391e-06 1.05137702e-02 1.31204710e-04
 4.47023735e-05 2.36825457e-03 4.52668933e-05 7.80520724e-03
 2.50126810e-04 3.79645018e-05 5.77848686e-06 5.79063042e-06
 3.20364953e-04 3.97295841e-06 7.17741178e-06 2.65589145e-04
 2.26465502e-05 5.98200213e-06 1.40228793e-03 1.67917892e-06
 8.75484936e-06 1.59140265e-05 1.48534554e-04 2.50662935e-05
 4.56018962e-03 2.34865663e-05 1.36756694e-05 1.35396547e-05
 3.65828847e-05 1.98024208e-06 3.59742947e-03 1.16728056e-04
 6.63503600e-05 2.13919608e-04 1.53530545e-05 3.65781732e-04
 7.63195870e-06 2.27548313e-06 5.52041845e-04 8.17747518e-06
 5.21123627e-04 0.00000000e+00 2.17781668e-05 2.22963893e-03
 4.08248234e-08 7.11545725e-05 3.67580183e-05 5.64053662e-04
 1.25613400e-04 4.51819755e-04 3.06186175e-07 1.02062058e-09
 1.02062058e-09 4.23957720e-06 2.81345242e-06 1.82293469e-05
 9.18558526e-09 1.478639

In [30]:
DTAssesment_dict = { "Feature_Name": filtered_df.iloc[: , 7:-1].columns, "Feature_Importance":Regressor.feature_importances_,}
DTAssesment_df = pd.DataFrame(DTAssesment_dict)
print(DTAssesment_df) 

                            Feature_Name  Feature_Importance
0                      reviews_per_month        3.814580e-02
1                               latitude        7.480044e-01
2                              longitude        1.378150e-01
3                            Shared room        2.255326e-04
4                        Entire home/apt        1.763045e-02
5                             Hotel room        7.888578e-05
6                           Private room        3.193735e-04
7                        Agincourt North        1.736697e-05
8           Agincourt South-Malvern West        1.140629e-04
9                              Alderwood        2.177324e-06
10                                 Annex        1.051377e-02
11                     Banbury-Don Mills        1.312047e-04
12                        Bathurst Manor        4.470237e-05
13                   Bay Street Corridor        2.368255e-03
14                       Bayview Village        4.526689e-05
15                 Bayvi

In [31]:
pd.set_option('display.max_rows', None)
DTAssesment_df.sort_values(by=['Feature_Name', 'Feature_Importance'])

,Feature_Name,Feature_Importance
7,Agincourt North,1.736697e-05
8,Agincourt South-Malvern West,1.140629e-04
9,Alderwood,2.177324e-06
10,Annex,1.051377e-02
11,Banbury-Don Mills,1.312047e-04
12,Bathurst Manor,4.470237e-05
13,Bay Street Corridor,2.368255e-03
14,Bayview Village,4.526689e-05
15,Bayview Woods-Steeles,7.805207e-03
16,Bedford Park-Nortown,2.501268e-04


# Definig dataset for the models

In the next lines, I am going to categorize the prices as low, affordable and expensive. Then then the model willl be trained sperately on each class.10 columns were selected to start with building our model and see how it works.

In [ ]:
def priceRanker(val):
    if  val< 100:
        return 1
    elif val < 200:
         return 2
    else :
         return 3

filtered_df["categoryPrice"] = df["price"].apply(priceRanker)
   

In [ ]:
filtered_df

There is one more step to have a clean data to start training our dataset. To disregard the outliers in high pricing, I am going to filter all the prices above 2000 dollars per night.

In [ ]:
new_df=filtered_df
new_df.sort_values('price') #Sorting is must

# making boolean series for DIS 
filter2 = new_df["price"]>2000
  
# filtering data on basis filter 
new_df.where(filter2, inplace = False) 
new_df

# SMOTE for imbalanced classification

if we go through the dataset after categorizing and applying the PriceRanker, it can be seen that there is an imblanced available data in mid range prices. To solve the problem we are going to deploy SMOTE.

In [ ]:
!pip install  imbalanced-learn
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
#define dataset
X_df = new_df.iloc[: , 7:-2]
X = X_df.values
y =new_df.iloc[:, -1].values
#Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.9, random_state = 0)
#SMOTE
sm = SMOTE()
X, y = sm.fit_sample(X_train, y_train)

# Decision Tree Cross Validation

For cross validation purposes, I am going to train 90% of our dataset.

---



now I need to define a new classifier to fit on our cross validation dataset. which I call it clf_cv

# Hyperparameter tuning for DT

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

acc_score_DT = []
best_score_DT = (0, -1)
best_clf_DT = None

for i in range(1,21):
    clf_DT = DecisionTreeClassifier(max_depth=i,  random_state=0)
    clf_DT.fit(X_train, y_train)
    y_pred_DT=clf_DT.predict(X_test)
    
    curr_acc_DT = accuracy_score(y_test, y_pred_DT)
    acc_score_DT.append((i, curr_acc_DT))
    
    if curr_acc_DT > best_score_DT[0]:
        best_score_DT = (curr_acc_DT, i)
        best_clf_DT = clf_DT

print(clf_DT)
acc_score_DT

In [ ]:

Price_Prediction = {"y_pred" : y_pred_DT, "y_test" :y_test}
dic_pred = pd.DataFrame(Price_Prediction)
dic_pred

## Matrix of confusion for Decision Tree cross validation

In [ ]:
from sklearn.metrics import confusion_matrix
clf_DT_final = DecisionTreeClassifier(max_depth=4,random_state=0)
clf_DT_final.fit(X_train, y_train)
y_pred_DT=clf_DT_final.predict(X_test)
cm_DT = confusion_matrix(y_test, y_pred_DT)
print(cm_DT)
accuracy_score(y_test, y_pred_DT)

# Random Forest Classification 


for random forest model we have to define the number of features that we are including in our model that for this case it is 147 ( total number of coulumns _ 9). 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Convert dataset from regression to classification
X, y = make_classification(n_samples=1000, n_features=147,
                            n_informative=3, n_redundant=0,
                            random_state=0, shuffle=False)


# Hyperparameter Tuning for RF model

#For tuning I am going to train and fit the model with a different values for max_depth and n_estimator. I chose a range of 3 to 20 for max_depth and 20 to 100 for n_estimator.

In [ ]:
acc_score_RF = []
best_score_RF = (0, -1)


# Create nested for loop
for i in range(6,20):
  for j in range(22,30):
    clf_RF = RandomForestClassifier(max_depth=i,n_estimators=j,random_state=0)
    clf_RF.fit(X_train, y_train)
    y_pred_RF=clf_RF.predict(X_test)
    
    curr_acc_RF = accuracy_score(y_test, y_pred_RF)
    acc_score_RF.append((i,j, curr_acc_RF))
    
    if curr_acc_RF > best_score_RF[0]:
        best_score_RF = (curr_acc_RF, i, j)
        best_clf_DT = clf_RF
        
print(best_clf_DT)



**bold text**# Now We need to creat a matrix of confusion for RF cross validation 

In [ ]:
clf_RF = RandomForestClassifier(max_depth=15,n_estimators=27,random_state=0)
clf_RF.fit(X_train, y_train)
y_pred_RF=clf_RF.predict(X_test)
cm_RF= confusion_matrix(y_test, y_pred_RF)
print(cm_RF)
accuracy_score(y_test, y_pred_RF)

# KNN Classifier

# Hyperparameter tuning for KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)


##The only difference in terms of model tuning for KNN is we are changing the number of nearest numbers for a range from 4 to 20.

In [ ]:
acc_score_knn = []
best_score_knn = (0, -1)

for i in range(14,35):
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train, y_train)
    y_pred_knn=neigh.predict(X_test) 
  
  
    
    curr_acc_knn = accuracy_score(y_test, y_pred_knn)
    acc_score_knn.append((i, curr_acc_knn))
    
    if curr_acc_knn > best_score_knn[0]:
        best_score_knn = (curr_acc_knn, i)
        best_clf_DT = neigh
print(best_clf_DT)
acc_score_knn

# KNN confusion matrix

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=14)
neigh.fit(X_train, y_train)
y_pred_knn=neigh.predict(X_test) 
cm_knn= confusion_matrix(y_test, y_pred_knn)
print(cm_knn)
accuracy_score(y_test, y_pred_knn)